**`Análisis exploratorio de los datos`**: _(Exploratory Data Analysis-EDA)_

Primero importamos las librerias que  vamos a necesitar

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

Cargamos los archivos .csv a los que ya se les aplico las transformaciones

In [3]:
df_amazon_prime = pd.read_csv('../dataset_ETL/amazon_prime_ETL.csv', delimiter = ',',encoding = "utf-8")
df_disney_plus = pd.read_csv('../dataset_ETL/disney_plus_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_hulu = pd.read_csv('../dataset_ETL/hulu_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_netflix = pd.read_csv('../dataset_ETL/netflix_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_ratings = pd.read_parquet('../dataset_ETL/ratings_ETL.csv')#, delimiter=',', encoding= "utf-8")

**Se corre un sample en cada dataframe para ver que tipo de datos hay**

In [ ]:
df_amazon_prime.sample(20)

In [ ]:
df_disney_plus.sample(20)

In [ ]:
df_hulu.sample(20)

In [ ]:
df_netflix.sample(20)

In [ ]:
df_ratings.sample(20)

Observamos la informacion detallada de cada dataframe

In [ ]:
df_amazon_prime.info()

In [ ]:
df_disney_plus.info()

In [ ]:
df_hulu.info()

In [ ]:
df_netflix.info()

In [15]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11024289 entries, 0 to 11024288
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userid     int64         
 1   scored     float64       
 2   timestamp  datetime64[ns]
 3   movieid    object        
 4   scoreds    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 420.5+ MB


Se analizaran los valores faltantes

In [10]:
df_amazon_prime.isnull().sum()

show_id             0
type                0
title               0
director         2082
cast             1233
country          8996
date_added       9513
release_year        0
rating              0
listed_in           0
description         0
id                  0
duration_int        0
duration_type       0
dtype: int64

In [12]:
df_disney_plus.isnull().sum()

show_id            0
type               0
title              0
director         473
cast             190
country          219
date_added         3
release_year       0
rating             0
listed_in          0
description        0
id                 0
duration_int       0
duration_type      0
dtype: int64

In [11]:
df_hulu.isnull().sum()

show_id             0
type                0
title               0
director         3070
cast             3073
country          1453
date_added         28
release_year        0
rating              0
listed_in           0
description         4
id                  0
duration_int        0
duration_type     479
dtype: int64

In [13]:
df_netflix.isnull().sum()

show_id             0
type                0
title               0
director         2634
cast              825
country           831
date_added         10
release_year        0
rating              0
listed_in           0
description         0
id                  0
duration_int        0
duration_type       3
dtype: int64

In [14]:
df_ratings.isnull().sum()

userid       0
scored       0
timestamp    0
movieid      0
scoreds      0
dtype: int64

**Se analizaran las columnas con valores numericos**

Como a la columna 'date_added' no lo toma con formato DATETIME, se convierte.

In [14]:
df_amazon_prime['date_added'] = pd.to_datetime(df_amazon_prime['date_added'])
df_disney_plus['date_added'] = pd.to_datetime(df_disney_plus['date_added'])
df_hulu['date_added'] = pd.to_datetime(df_hulu['date_added'])
df_netflix['date_added'] = pd.to_datetime(df_netflix['date_added'])

In [12]:
df_amazon_prime['date_added'].min()

Timestamp('2021-03-30 00:00:00')

In [15]:
df_disney_plus['date_added'].min()

Timestamp('2019-10-01 00:00:00')

In [16]:
df_hulu['date_added'].min()

Timestamp('2006-07-14 00:00:00')

In [6]:
df_ratings['timestamp'].min()

Timestamp('1995-01-09 00:00:00')

**Se analizaran si hay duplicados**

In [4]:
df_ratings.duplicated().value_counts()

False    11013823
True        10466
dtype: int64

**Se identificaran valores atípicos en la columna "scored" del dataframe ratings**

In [8]:
q1 = df_ratings['scored'].quantile(0.25)
q3 = df_ratings['scored'].quantile(0.75)
iqr = q3 - q1

# define los límites para los outliers
lower_limit = q1 - 1.5 * iqr
upper_limit = q3 + 1.5 * iqr

# identifica los outliers
outliers = df_ratings[(df_ratings['scored'] < lower_limit) | (df_ratings['scored'] > upper_limit)]

# muestra los outliers
print(outliers)

          userid  scored  timestamp movieid  scoreds
0              1     1.0 2015-03-09   as680      3.5
24             1     0.5 2015-03-09  as7193      3.5
42             2     1.0 1997-06-23  hs1914      3.6
43             2     1.0 1997-06-23  as9455      3.5
84             4     1.0 2003-01-15  ns3279      3.6
...          ...     ...        ...     ...      ...
11024167  124373     1.0 2001-05-31  ns7039      3.5
11024171  124374     1.0 2001-11-28  ds1361      3.6
11024179  124374     1.0 2001-11-28  as9146      3.6
11024192  124375     1.0 1996-08-14   hs282      3.4
11024218  124376     1.0 2001-04-26  ns5214      3.6

[514764 rows x 5 columns]


El resultado que te devolvió es un subconjunto de datos de la tabla. En este subconjunto, se han eliminado todas las filas donde los valores de la columna "scored" se encuentran fuera del rango intercuartil, lo que sugiere que son posibles valores atípicos o "outliers". En otras palabras, estos son los datos que no son atípicos en la columna "scored". La tabla resultante tiene 514764 filas y 5 columnas. Cada fila contiene información de un usuario que ha calificado una película, incluyendo el ID del usuario, su calificación ("scored"), la fecha en que se hizo la calificación ("timestamp"), el ID de la película ("movieid") y el promedio de todas las calificaciones para esa película ("scored").

**CONCLUSIONES EDA**

GENERALES (considerando el objetivo y alcance de este proyecto):

- Las columnas duration tienen tanto información en minutos como de temporadas

- Las columnas cast tienen los actores separados por una coma

PARTICULARES

1- df_amazon_prime_ETL

- separdor ','+ encoding utf-8
- cararcteres especiales [ (30)
- valores faltantes
    + director   2082
    + cast       1233
    + country    8996
    + date_added 9513
- 9687 registros
- 12 columnas

2- df_disney_plus_ETL

- separdor ','+ encoding utf-8
- cararcteres especiales [ (NO)
- valores faltantes
    + director      473
    + cast          190
    + country       219
    + date_added    3
- 1450 registros
- 12 columnas


3- df_hulu_ETL

- separdor ','+ encoding utf-8
- cararcteres especiales [ (1)
- valores faltantes
    + director      3070
    + cast          3073
    + country       1453
    + date_added    28
    + description   4
    + duration_type 479
- 3090 registros
- 12 columnas

4- df_netflix_ETL

- valores faltantes
    + director 2634
    + cast 825
    + country 831
    + date_added 10
    + duration 3
- 8807 registros
- 12 columnas

5- df_rating_ETL
-valores faltantes 0
- 11024289 registros
- 5 columnas
- duplicados 10466

Al obtener el primer año de ingreso de la columna 'timestamp' (1995-01-09),se concluyo que es un valor fuera de lo normal dado que las plataformas abrieron años depues. 